In [103]:
import numpy as np
import sys
import os
import codecs

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop, Adam

from keras.callbacks import ModelCheckpoint, LambdaCallback
from keras.utils import np_utils

import tensorflow as tf

In [16]:
# load ascii text and covert to lowercase
filenames = [f for f in os.listdir('data') if f.startswith("Harry")]
raw_text = "\n".join(codecs.open(os.path.join('data', f), 'rb', 'cp1252', errors='ignore').read() for f in filenames)
#raw_text = "\n".join(open('data//' + f).read() for f in filenames)
raw_text = raw_text.lower()

In [18]:
print(len(raw_text))
raw_text[10000:14000]

6321551


'with... you know... her crowd." \r\n\r\nmrs. dursley sipped her tea through pursed lips. mr. dursley wondered whether he dared tell her he\'d heard the name "potter." he decided he didn\'t dare. instead he said, as casually as he could, "their son -- he\'d be about dudley\'s age now, wouldn\'t he?" \r\n\r\n"i suppose so," said mrs. dursley stiffly. \r\n\r\n"what\'s his name again? howard, isn\'t it?" \r\n\r\n"harry. nasty, common name, if you ask me." \r\n\r\n"oh, yes," said mr. dursley, his heart sinking horribly. "yes, i quite agree." \r\n\r\nhe didn\'t say another word on the subject as they went upstairs to bed. \r\n\r\nwhile mrs. dursley was in the bathroom, mr. dursley crept to the bedroom window and peered down into the front garden. the cat was still there. \r\n\r\nit was staring down privet drive as though it were waiting for something. \r\n\r\nwas he imagining things? could all this have anything to do with the potters? if it did... if it got out that they were related to a 

In [19]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Unique chars: ", n_vocab)

Total Characters:  6321551
Total Unique chars:  78


In [20]:
print(chars)

['\n', '\r', '\x1f', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '¦', '«', '\xad', '»', 'é', 'ü', '–', '‘', '’', '“', '•']


In [131]:
# create input and output pairs
seq_length = 64
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total sequences: ", n_patterns)

Total sequences:  6321487


In [132]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length))
print(X.shape)
X[:2, :]
# normalize
#X = X / float(n_vocab)

(6321487, 64)


array([[45, 38, 55, 55, 62,  3, 53, 52, 57, 57, 42, 55,  3, 38, 51, 41,  3,
        57, 45, 42,  3, 56, 52, 55, 40, 42, 55, 42, 55,  9, 56,  3, 56, 57,
        52, 51, 42,  3,  1,  0,  1,  0, 40, 45, 38, 53, 57, 42, 55,  3, 52,
        51, 42,  3,  1,  0,  1,  0, 57, 45, 42,  3, 39, 52],
       [38, 55, 55, 62,  3, 53, 52, 57, 57, 42, 55,  3, 38, 51, 41,  3, 57,
        45, 42,  3, 56, 52, 55, 40, 42, 55, 42, 55,  9, 56,  3, 56, 57, 52,
        51, 42,  3,  1,  0,  1,  0, 40, 45, 38, 53, 57, 42, 55,  3, 52, 51,
        42,  3,  1,  0,  1,  0, 57, 45, 42,  3, 39, 52, 62]])

In [133]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
seed = dataX[start]
print(pattern)
print("Seed pattern:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("Next letter: %s"%(int_to_char[dataY[start]]))

[53, 46, 41, 42, 55, 13, 3, 47, 42, 55, 48, 46, 51, 44, 3, 38, 51, 41, 3, 57, 60, 46, 57, 40, 45, 46, 51, 44, 15, 1, 0, 1, 0, 5, 38, 51, 57, 46, 41, 52, 57, 42, 56, 4, 5, 3, 56, 38, 46, 41, 3, 56, 51, 38, 53, 42, 13, 3, 49, 52, 52, 48, 46, 51]
Seed pattern:
" pider, jerking and twitching.

"antidotes!" said snape, lookin "
Next letter: g


In [134]:
# one hot encode the output variable
#y = np_utils.to_categorical(dataY)
y = np.array(dataY)
print(y.shape)
y[:5]
#WE USE SPARSE CATEGORICAL

(6321487,)


array([62,  3, 60, 45, 52])

In [149]:
# define the input shape
hidden_size=512

inputs = Input(shape=(X.shape[1],))
embeded = Embedding(len(chars), hidden_size, input_length=seq_length)(inputs)
# x = LSTM(hidden_size, return_sequences = True)(embeded) 
# x = Dropout(0.2)(x)
# x = Bidirectional(LSTM(hidden_size, recurrent_dropout=0.2, return_sequences=True))(embeded)
# x = Bidirectional(LSTM(hidden_size, recurrent_dropout=0.3))(x)
x = LSTM(hidden_size, return_sequences = True, recurrent_dropout=0.2)(embeded)
x = Dropout(0.3)(x)
x = LSTM(hidden_size, return_sequences = True, recurrent_dropout=0.3)(x)
x = Dropout(0.4)(x)
x = LSTM(hidden_size, recurrent_dropout=0.4)(x)
x = Dropout(0.5)(x)
output = Dense(len(chars), activation ='softmax')(x)

model = Model(inputs, output)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 64)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 64, 512)           39936     
_________________________________________________________________
lstm_21 (LSTM)               (None, 64, 512)           2099200   
_________________________________________________________________
dropout_18 (Dropout)         (None, 64, 512)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 64, 512)           2099200   
_________________________________________________________________
dropout_19 (Dropout)         (None, 64, 512)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 512)               2099200   
__________

In [167]:
def sample(preds, temperature=0.4):
    # helper function to sample an index from a probability array
    # Taken from keras text generation example
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, np.squeeze(preds), 1)
    return np.argmax(probas)


def generate_stuff(length=200):
    # pick a random seed
    start = np.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    seed = dataX[start]
    #print(pattern)
    print("Seed pattern:")
    print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    
    generated_text = []
    # generate characters
    for i in range(length):
        x = np.reshape(pattern, (1, len(pattern)))
        prediction = model.predict(x, verbose=0)
        index = sample(prediction) #Use sampling instead
        #index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        pattern.append(index)
        generated_text.append(index)
        pattern = pattern[1:]
        #print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    #print("\nDone.")
    #print(pattern)
    #print("\"", ''.join([int_to_char[value] for value in seed]), "\"")
    print("Generated:")
    print("\"", ''.join([int_to_char[value] for value in generated_text]), "\"")
    
generate_stuff(2000)

Seed pattern:
" lat side of silver dagger,
 "
Generated:
" 
"yes," said harry. "i can't think the part of the horse and good to see you at the staircase of a silver door of harry and the table of the stairs was something on his mouth from the stairs of the books and the prophet of his face was he was a ground of the table of the rather school parchment in the high bottom of the beater and now and the teachers were staring at him as he had the stand and come up his tone and say in the stairs to the corridor and holding the door and she had been going to be a forest than he had seen the ground, he had not been here and the chair had reading a few back of the start of his wand. harry was something with a high black moment and still stopped to him and harry stared in the man and the distonite disappeared and patched by the completely in his story and concentrated to harry from the wizard to the stairs and found the stone bagman of the dark side of the death eaters and whispered that the de

In [158]:
model = load_model("Harrypotter_Gen_exp04_02.h5")

In [163]:
opt = Adam(lr=0.0001)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt)

In [165]:
#Train!

# define the checkpoint
#filepath="checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}-gentext-CharRNN-simple.hdf5"
epoch_print = LambdaCallback(
    on_epoch_end=lambda batch,logs: generate_stuff())
checkpoint = ModelCheckpoint('Harrypotter_Gen_exp04_02.h5', monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, epoch_print]

model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
6321472/6321487 [============================>.] - ETA: 0s - loss: 1.4926Epoch 00000: loss improved from inf to 1.49260, saving model to Harrypotter_Gen_exp04_02.h5
Seed pattern:
" ght filled the prison room and the frail old body was lifted fro "
Generated:
" m the wizards shouting the ground and stepped at the books and summer in the boy, harry and hermione was a door and really while they were a death from the door, he was straightening toward the table  "
6321487/6321487 [==============================] - 16212s - loss: 1.4926 
Epoch 2/20
6321472/6321487 [============================>.] - ETA: 0s - loss: 1.4109Epoch 00001: loss improved from 1.49260 to 1.41092, saving model to Harrypotter_Gen_exp04_02.h5
Seed pattern:
" rrying, bill pulled the door behind him closed, shutting out the "
Generated:
"  street of the stairs and he was a table of his boy and his face was saying his feet to a gringott of his confidence and the sight of the still was sirius to the leading of hi

KeyboardInterrupt: 

In [141]:
model.save('Harrypotter_Gen_exp03.h5')


In [127]:
# model = load_model("Harrypotter_Gen.h5")
# generate_stuff()

[48, 46, 51, 56, 13, 3, 45, 42, 3, 49, 52, 52, 48, 42, 41, 3, 56, 52, 3, 45, 38, 51, 41, 56, 52, 50, 42, 3, 38, 51, 41, 3]
Seed pattern:
" kins, he looked so handsome and  "


ValueError: Error when checking : expected input_4 to have shape (None, 64) but got array with shape (1, 32)


Done.
[41, 3, 57, 52, 3, 57, 45, 42, 3, 56, 57, 38, 55, 57, 42, 41, 3, 57, 52, 3, 57, 45, 42, 3, 56, 57, 38, 55, 57, 42, 41, 3, 57, 52, 3, 57, 45, 42, 3, 56, 57, 38, 55, 57, 42, 41, 3, 57, 52, 3, 57, 45, 42, 3, 56, 57, 38, 55, 57, 42, 41, 3, 57, 52]
"  until proven guilty, severus," he said firmly.
snape looked fur "
" started to the started to the started to the started to the started to the started to the started to "
